# Python

## Task -1

### Func

In [1]:
def stir_not_shake(recipe, ingredients):
  # Проверка, есть ли все ингредиенты
  for ingredient, amount in recipe.items():
    if ingredient not in ingredients or ingredients[ingredient] < amount:
      return 0

  # Определение количества коктейлей
  min_amount = float('inf')
  for ingredient, amount in recipe.items():
    if ingredients[ingredient] / amount < min_amount:
      min_amount = ingredients[ingredient] / amount

  return int(min_amount)

### Check (examples)

In [2]:
num_cocktails = stir_not_shake(
    {'bourbon': 50, 'vermouth': 25, 'angostura': 1, 'cherry': 1},
    {'bourbon': 500, 'vermouth': 750, 'angostura': 150, 'cherry': 2, 'eggs': 20}
)

print(f"Иван может сделать {num_cocktails} коктейлей.")

Иван может сделать 2 коктейлей.


In [3]:
num_cocktails = stir_not_shake(
    {'prosecco': 90, 'orange_juice': 90},
    {'prosecco': 1500, 'campari': 750, 'vermouth': 750}
)
print(f"Иван может сделать {num_cocktails} коктейлей.")

Иван может сделать 0 коктейлей.


## Task - 2

In [4]:
'''
# Импорт необходимых библиотек
import datetime    # Для работы с форматами даты и времени
import pandas as pd     # Для работы с данными (предобработка, анализ)
import clickhouse_driver    # Для подключения к СУБД ClickHouse
import plotly.express as px     # Для визуализации
import getpass      # Для логина в БД

# Подключение к ClickHouse (ввод логина и пароля)
host_name = f'{ваш_хост}'
user_name = f'{ваше_имя_пользователя}'
password = getpass.getpass(prompt='Password: ')
db = f'{ваша_база_данных}'

# Подключение к ClickHouse
ch = clickhouse_driver.Client(host=host_name, user=user_name, password=password, database=db)

# Объявление переменной и присвоение ей результата SQL-запроса
ilp_query = """
select *
from geo_comp_list
where 1=1
    and dt = (select max(dt) from geo_comp_list) -- Получение данных только за последнюю дату
    and panel = 'iLP'  -- Фильтрация по панели 'iLP'
"""

# Выполнение запроса и сохранение результатов в DataFrame
df_ilp = ch.query_dataframe(ilp_query)

# Загрузка данных из двух отдельных CSV-файлов
df_pm = pd.read_csv('geo-data/geo-pm.csv', sep=';')
df_scp = pd.read_csv('geo-data/geo-scp.csv', sep=';')

# Объединение данных из разных источников в один DataFrame
df = pd.concat([df_pm, df_scp, df_ilp], ignore_index=True)

# Получение максимальной даты в данных
dt = df['dt'].max()

# Фильтрация данных: оставляются только записи с comparison == False
br = df[df['comparison'] == False]

# Экспорт отфильтрованных данных в Excel-файл
filename = f'C:\\Users\\Workwork\\Documents\\Panel\\{str(dt)}-bad-region.xlsx'
br.to_excel(filename)

# Присоединение данных о регионах из другого DataFrame (предполагается, что df_scp имеет столбцы georegionid, city_town_locality, district, state и federal_area)
# методом merge по столбцам 'anc_id' и 'georegionid'
br_anc = br.merge(
    rh[['georegionid','city_town_locality', 'district', 'state', 'federal_area']],
    left_on='anc_id',
    right_on='georegionid'
)

# Группировка данных по столбцам и агрегация количества (count) уникальных логинов (login)
tree_anc = br_anc.groupby(['federal_area', 'state', 'recrout_type', 'panel'])\
    .agg(cnt=('login','nunique')).reset_index()

# Создание визуализации TreeMap с помощью Plotly Express
fig = px.treemap(
    tree_anc,
    path=['federal_area', 'recrout_type', 'panel'],
    values='cnt',
    title="Разница в географии, регионы из анкеты"
)

# Настройка внешнего вида визуализации
fig.data[0].textinfo = 'label+text+value'
fig.layout.hovermode = False

# Отображение визуализации
fig.show()
'''

'\n# Импорт необходимых библиотек\nimport datetime    # Для работы с форматами даты и времени\nimport pandas as pd     # Для работы с данными (предобработка, анализ)\nimport clickhouse_driver    # Для подключения к СУБД ClickHouse\nimport plotly.express as px     # Для визуализации\nimport getpass      # Для логина в БД\n\n# Подключение к ClickHouse (ввод логина и пароля)\nhost_name = f\'{ваш_хост}\'\nuser_name = f\'{ваше_имя_пользователя}\'\npassword = getpass.getpass(prompt=\'Password: \')\ndb = f\'{ваша_база_данных}\'\n\n# Подключение к ClickHouse\nch = clickhouse_driver.Client(host=host_name, user=user_name, password=password, database=db)\n\n# Объявление переменной и присвоение ей результата SQL-запроса\nilp_query = """\nselect *\nfrom geo_comp_list\nwhere 1=1\n    and dt = (select max(dt) from geo_comp_list) -- Получение данных только за последнюю дату\n    and panel = \'iLP\'  -- Фильтрация по панели \'iLP\'\n"""\n\n# Выполнение запроса и сохранение результатов в DataFrame\ndf_i

Этот скрипт нужен для:
- получения данных из БД ClickHouse
- подгрузки двух CSV-файлов и объединения их с результатом запроса их CH
- Фильтрации данных и экспорт в Excel
- визуализации

# SQL

In [5]:
'''
Здесь происходит выполнение SQL запроса на основе данных из таблицы osnova_d и activity_d с использованием Apache Spark
где для каждой записи из osnova_d вычисляются различные метрики, связанные с активностью участников исследования.

-- Объявление переменной и присвоение ей результата запроса
val osnova_d_activity = spark.sql ("""
    SELECT 
        osnova_d.*,
                                   
        -- Рассчитываем стаж (Experience) как разницу в датах между полями Data и DateFrom.
        DATEDIFF(Data, DateFrom) AS Experience,
                                   
        -- Рассчитываем количество активных участников за последние 30 дней как Active30.
        MAX(CASE WHEN a.researchdate BETWEEN DATE_ADD(Data, -29) AND Data THEN a.SubjectID END) AS Active30,
                                   
        -- Рассчитываем количество активных участников за последние 30 дней, исключая системных как Active30_not_system.
        MAX(CASE WHEN a.researchdate BETWEEN DATE_ADD(Data, -29) AND Data THEN a.SubjectID END) AS Active30_not_system,
                                   
        -- Рассчитываем количество активных участников в текущем дне как Active0
        MAX(CASE WHEN a.researchdate = Data THEN a.SubjectID END) AS Active0,
                                   
        -- Рассчитываем количество активных участников в текущем дне, исключая системных как Active0_not_system
        MAX(CASE WHEN a.researchdate = Data THEN a.SubjectID END) AS Active0_not_system,
                                   
        -- Рассчитываем количество дней, когда участник был активным за последние 30 дней как Active30_days_in_month,
        -- группируя по участнику и месяцу.
        COUNT(MAX(CASE WHEN a.researchdate BETWEEN DATE_ADD(Data, -29) AND Data THEN a.SubjectID END)) OVER (PARTITION BY osnova_d.SubjectID, Month) AS Active30_days_in_month,
                                   
        -- Рассчитываем процент активности (округляя) за последние 30 дней от общего числа дней в месяце как Month_activity30_percent,
        -- группируя по участнику и месяцу.
        ROUND(COUNT(MAX(CASE WHEN a.researchdate BETWEEN DATE_ADD(Data, -29) AND Data THEN a.SubjectID END)) OVER (PARTITION BY osnova_d.SubjectID, Month) / MAX(DAY(Data)) OVER (PARTITION BY Month), 2) AS Month_activity30_percent
    
    -- Таблица, из которой берем данные
    FROM 
        osnova_d
    
    -- Присоединяем к ней др таблицу методом left join (к левой таблице присоединяем правую)
    LEFT JOIN 
        activity_d a
    
    -- Определяем ключевой столбец по которому происходит объединение
    ON 
        osnova_d.SubjectID = a.SubjectID

    -- Группировка (выполняется последовательно слева направо)
    GROUP BY 
        Data, osnova_d.Month, osnova_d.SubjectID, Panel, Recrout_type, Research_type, DateFrom, DateTo, In, Out
""")
 Записываем результаты в файл "osnova_d_activity".
osnova_d_activity.write.mode("overwrite").parquet(Path.concat("osnova_d_activity"))
'''


Этот скрипт нужен для анализа активности участников исследования на основе данных из двух таблиц "osnova_d" и "activity_d". Он выполняет несколько расчетов и агрегаций, чтобы получить информацию о стаже участников, количестве активных участников за последние 30 дней, их дневной активности и другие метрики. Результаты анализа сохраняются в parquet-файл для дальнейшего анализа.